<a href="https://colab.research.google.com/github/itismeganrms/rl-a3/blob/main/Code/reinforce_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wandb --quiet
!pip install swig --quiet
!pip install gym[all] --quiet
!pip install pygame --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.8/266.8 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.0/297.0 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 22.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 41.8 MB/s eta 0:

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [14]:
## fixing the pygame error

import os
os.environ['SDL_VIDEODRIVER']='dummy'
import pygame
pygame.display.set_mode((640,480))

<Surface(640x480x32 SW)>

In [15]:
import gym
from gym.envs import box2d
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [17]:
class Policy_NN(nn.Module):
    def __init__(self, input_size, output_size, hidden_size = 128, gamma = 0.99, learning_rate = 0.001):
        super().__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_size = hidden_size
        self.gamma = gamma
        self.learning_rate = learning_rate
        self.layer1 = nn.Linear(input_size, hidden_size)
        self.layer2 = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=-1)

        self.optimizer = optim.Adam(self.parameters(),lr=self.learning_rate)


    def forward(self,x):
      x = self.layer1(x) #forward propagation
      x = self.layer2(x)
      x = self.softmax(x)
      return x

    def act(self,state):
        #With obs, find the maximum Q value and output the corresponding action
        #action_probs = torch.softmax(self.actor(obs), dim=-1)
        #state_value = self.critic(state)

        #obs_tensor = torch.as_tensor(obs, dtype=torch.float32)
        #q_value = self(obs_tensor.unsqueeze(0)) #convert to row vector
        #highest_q_value = torch.argmax(input=q_value)
        #log_value = torch.log(q_value.squeeze(0)[highest_q_value])
        #action = log_value.detach().item() #action corresponding to max Q index
        #return action, log_value
        #print("checkpoint 6")
        action_probs = torch.softmax(self.policy_network(state), dim=-1)
        #print("checkpoint 7")
        #print("action_probs: {}".format(action_probs))
        #state_value = self.critic(state)
        return action_probs

In [30]:
class reinforce_update:
    def __init__(self, Policy_NN, rewards, log_value):
        self.network = Policy_NN
        self.rewards = rewards
        self.log_value = log_probs_array

    def update_function(self, Policy_NN, rewards, log_probs_array):
        discount_rewards = []
        Gt,pw = 0, 0
        for r in reversed(rewards):
            Gt = Gt + gamma**pw * r
            pw = pw + 1
            discount_rewards.append(Gt)

        steps_value = np.arange(rewards.size)

        discount_rewards = torch.tensor(discount_rewards)
        discount_rewards = discount_rewards[::-1].cumsum()[::-1] / gamma**steps_value

        # def discount_rewards(rewards, gamma):
        # t_steps = np.arange(rewards.size)
        # r = rewards * gamma**t_steps
        # r = r[::-1].cumsum()[::-1] / gamma**t_steps
        # return r

        policy_gradient = []
        for log_prob, e_d_r in zip(self.log_value, discount_rewards):
            policy_gradient.append(-log_prob*e_d_r)

        policy_network.optimizer.zero_grad()
        policy_gradient = torch.stack(policy_gradient).sum()
        policy_gradient.backward()
        policy_network.optimizer.step()

        return policy_gradient

In [7]:
def update(policy_network, gamma = 0.99):
    log_probs = []
    rewards = []

In [8]:
import torch
import numpy as np
import gym
import statistics

In [32]:
def main():
    env = gym.make("LunarLander-v2")
    state_size = env.observation_space.shape[0]
    action_size = env.action_space.n
    policy_network = Policy_NN(state_size, action_size)

    episode_array, rewards_array = [] , []
    n_episode = 500
    n_timesteps = 1000

    Reward_list = np.empty(shape=n_episode)

    state = env.reset()

    for episode in range(n_episode):
        log_probs_array,rewards = [], []
        total_reward = 0
        for step in range(n_timesteps):
            stateT = torch.tensor(state, dtype=torch.float32).unsqueeze(0)
            action_probs = policy_network(stateT)
            action_dist = torch.distributions.Categorical(action_probs)
            action = action_dist.sample()
            log_prob = action_dist.log_prob(action.unsqueeze(0))
            next_state, reward, done, _ = env.step(action.item())
            print("Episode:{} |  Step: {} | action: {} | Reward: {}".format(episode, step,action.item(),reward))
            if done:
                reinforce_update.update_function(policy_network, rewards,log_probs_array)
                rewards_array.append(np.sum(rewards))
                s = env.reset()
            else:
                s = next_state
            # total_reward += reward
            # rewards.append(reward)
            # state = next_state

    discounted_rewards = []
    R = 0
    for reward in rewards:
        R = reward + policy_network.gamma * R
        discounted_rewards.insert(0,R)

    # Compute policy loss
    policy_loss = torch.stack(log_probs) * discounted_rewards
    policy_loss = -policy_loss.sum()

    # Update policy
    optimizer.zero_grad()
    policy_loss.backward()
    optimizer.step()

In [33]:
main()

Episode:0 |  Step: 0 | action: 3 | Reward: -2.1968751823225134
Episode:0 |  Step: 1 | action: 3 | Reward: -2.39859730875571
Episode:0 |  Step: 2 | action: 1 | Reward: -1.8057036958187769
Episode:0 |  Step: 3 | action: 3 | Reward: -2.2393397397835018
Episode:0 |  Step: 4 | action: 2 | Reward: 4.240361241312127
Episode:0 |  Step: 5 | action: 0 | Reward: -2.0035647890476582
Episode:0 |  Step: 6 | action: 0 | Reward: -1.9464367181026319
Episode:0 |  Step: 7 | action: 1 | Reward: -1.6270788406020233
Episode:0 |  Step: 8 | action: 2 | Reward: 3.8309755061683974
Episode:0 |  Step: 9 | action: 3 | Reward: -2.221772831807441
Episode:0 |  Step: 10 | action: 0 | Reward: -1.8547247300237188
Episode:0 |  Step: 11 | action: 1 | Reward: -1.5624170571761613
Episode:0 |  Step: 12 | action: 2 | Reward: 3.493442737919179
Episode:0 |  Step: 13 | action: 1 | Reward: -1.2379095756786274
Episode:0 |  Step: 14 | action: 3 | Reward: -1.8140115442231763
Episode:0 |  Step: 15 | action: 3 | Reward: -2.08212322801

TypeError: reinforce_update.update_function() missing 1 required positional argument: 'log_probs_array'

In [12]:
x = episode_array
y = rewards_array
#plotting the comparison between the two
plt.title("Performance of REINFORCE")
plt.plot(x, y, label = "REINFORCE")
plt.xlabel("Episodes")
plt.ylabel("Rewards")
plt.legend()
plt.show()

NameError: name 'episode_array' is not defined